In [2]:
DATE = "2021-08-22"
TASK = "iterative-correction-via-resampling"
DIR = mkpath("$(homedir())/$(DATE)")

"/Users/cameronprybol/2021-08-22"

In [62]:
import Pkg
pkgs = [
#     "BioAlignments",
    "BioSequences",
#     "Clustering",
#     "CodecZlib",
#     "Colors",
#     "Combinatorics",
#     "DataFrames",
#     "DataStructures",
#     "Dates",
#     "DelimitedFiles",
#     "Distances",
#     "EzXML",
    "FASTX",
#     "GFF3",
#     "HTTP",
#     "Impute",
#     "JSON",
    "LightGraphs",
#     "Measures",
    "MetaGraphs",
    "https://github.com/cjprybol/Mycelia.git",
#     "NumericIO",
#     "PlotlyJS",
#     "Plots",
    "Primes",
#     "Printf",
#     "ProgressMeter",
    "Random",
#     "Revise",
#     "Statistics",
    "StatsBase",
#     "StatsPlots",
#     "StringDistances",
#     "uCSV",
#     "XLSX",
]

unregistered_packages = filter(pkg -> occursin(r"(^https|git$)", pkg), pkgs)
registered_packages = setdiff(pkgs, unregistered_packages)

for pkg in registered_packages
    try
        eval(Meta.parse("import $(pkg)"))
    catch
        Pkg.add(pkg)
        Pkg.build(pkg)
        eval(Meta.parse("import $(pkg)"))
    end
end

for pkg_url in unregistered_packages
    pkg_name = replace(basename(pkg_url), ".git" => "")
    try
        eval(Meta.parse("import $(pkg_name)"))
    catch
        Pkg.develop(url=pkg_url)
        Pkg.build(pkg_name)
        eval(Meta.parse("import $(pkg_name)"))
    end
end

In [4]:
function assess_kmer_sparsity_in_reads(k, observations)
#     @show k, observations
    canonical_kmer_counts = Mycelia.count_canonical_kmers(BioSequences.DNAMer{k}, observations)
#     canonical_kmer_counts = Mycelia.count_canonical_kmers(BioSequences.DNAMer{k}, first(observations))
# #     @show canonical_kmer_counts
#     for i in 2:length(observations)
#         canonical_kmer_counts_ = Mycelia.count_canonical_kmers(BioSequences.DNAMer{k}, observations[i])
#         canonical_kmer_counts = merge!(+, canonical_kmer_counts, canonical_kmer_counts_)
#     end
#     canonical_kmer_counts = map(o -> Mycelia.count_canonical_kmers(BioSequences.DNAMer{k}, o), observations)...)
    total_observed_canonical_kmers = length(keys(canonical_kmer_counts))
    total_possible_canonical_kmers = (4^k)/2
    sparsity = round(total_observed_canonical_kmers/total_possible_canonical_kmers*100, digits=5)
    @show sparsity
    return sparsity
end

assess_kmer_sparsity_in_reads (generic function with 1 method)

In [5]:
# generate a genome

In [6]:
sequence_length = 10
Random.seed!(sequence_length)
sequence = BioSequences.randdnaseq(sequence_length)
sequence_id = string(hash(sequence))
record = FASTX.FASTA.Record(sequence_id, sequence)

FASTX.FASTA.Record:
   identifier: 13030913541223777560
  description: <missing>
     sequence: CAGCCGACGG

In [7]:
# randomly sample reads with errors from the genome

In [8]:
?FASTX.FASTQ.Record

```
LSHFunction(similarity, args...; kws...)
```

Construct the default `LSHFunction` subtype that corresponds to the similarity function `similarity`.

# Arguments

  * `similarity`: the similarity function you want to use. Can be any of the following:

      * `cossim`
      * `inner_prod`
      * `jaccard`
      * `ℓ1`
      * `ℓ2`
  * `args...`: arguments to pass on to the default `LSHFunction` constructor corresponding to `similarity`.
  * `kws...`: keyword parameters to pass on to the default `LSHFunction` constructor corresponding to `similarity`.

# Returns

Returns a subtype of `LSH.LSHFunction` that hashes the similarity function `similarity`.

# Examples

In the snippet below, we construct `LSHFunctions.SimHash` (the default hash function corresponding to cosine similarity) using `LSHFunction()`:

```jldoctest; setup = :(using LSHFunctions)
julia> hashfn = LSHFunction(cossim);

julia> typeof(hashfn) <: LSHFunctions.SimHash <: LSHFunction
true
```

We can provide arguments and keyword parameters corresponding to the hash function that we construct:

```jldoctest; setup = :(using LSHFunctions)
julia> hashfn = LSHFunction(inner_prod, 100; dtype=Float64, maxnorm=10);

julia> n_hashes(hashfn) == 100 &&
       typeof(hashfn) <: SignALSH{Float64} &&
       hashfn.maxnorm == 10
true
```

See also: [`lsh_family`](@ref)


In [11]:
# 1%
error_rate = 0.01
n_reads = 100

# observations = [Mycelia.observe(FASTX.sequence(record), error_rate = error_rate) for i in 1:n_reads]
observations = [Mycelia.observe(record, error_rate = error_rate) for i in 1:n_reads]

100-element Vector{FASTX.FASTQ.Record}:
 FASTX.FASTQ.Record:
   identifier: 14129865098702156069
  description: 13030913541223777560
     sequence: CCGTCGGCTG
      quality: UInt8[0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c]
 FASTX.FASTQ.Record:
   identifier: 13030913541223777560
  description: 13030913541223777560
     sequence: CAGCCGACGG
      quality: UInt8[0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c]
 FASTX.FASTQ.Record:
   identifier: 13030913541223777560
  description: 13030913541223777560
     sequence: CAGCCGACGG
      quality: UInt8[0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c]
 FASTX.FASTQ.Record:
   identifier: 367592803644673128
  description: 13030913541223777560
     sequence: CAGCCGATCGG
      quality: UInt8[0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c]
 FASTX.FASTQ.Record:
   identifier: 13030913541223777560
  description: 13030913541223777560
     sequence: CAGCCGACGG
      quality: UInt8[0x3c, 0x3c, 0x3

In [12]:
# build assembly graph

In [13]:
k_options = Primes.primes(3, 31)
k_index = findfirst(k -> assess_kmer_sparsity_in_reads(k, observations) < 100, k_options)
k = k_options[k_index]

sparsity = 53.125


3

In [14]:
observations_file = "$(DIR)/LENGTH-$(sequence_length)-DEPTH-$(n_reads).fastq"
open(observations_file, "w") do io
    fastq_io = FASTX.FASTQ.Writer(io)
    for record in observations
        write(fastq_io, record)
    end
    close(fastq_io)
end

In [15]:
observation_graph = Mycelia.fastx_to_simple_kmer_graph(BioSequences.DNAMer{k}, observations_file)

┌ Info: creating graph
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:3001


{17, 48} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [25]:
# visualize
gfa_file = observations_file * ".k-$k.gfa"
Mycelia.graph_to_gfa(observation_graph, gfa_file)

run(`/Applications/Bandage.app/Contents/MacOS/Bandage image $(gfa_file) $(gfa_file).svg --depwidth .9 --deppower .9`)
# --nodewidth <float> Average node width (0.5 to 1000, default: 5)
# --depwidth <float>  Depth effect on width (0 to 1, default: 0.5)
# --deppower <float>  Power of depth effect on width (0 to 1, default: 0.5)

html_path_to_svg = "./" * repeat("../", length(split(pwd(), '/')) - 3)
html_path_to_svg *= replace("$(gfa_file).svg", "$(homedir())/" => "")

x = "<img src=$(html_path_to_svg)>"
display("text/html", x)
# display("image/svg+xml", read("$(gfa_file).svg", String))

In [ ]:
# for each read

In [27]:
observation = first(observations)

FASTX.FASTQ.Record:
   identifier: 14129865098702156069
  description: 13030913541223777560
     sequence: CCGTCGGCTG
      quality: UInt8[0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c, 0x3c]

In [45]:
kmer_weights = Dict(observation_graph.vprops[v][:kmer] => observation_graph.vprops[v][:weight] for v in LightGraphs.vertices(observation_graph))

total_observed_kmers = sum(values(kmer_weights))

kmer_probabilities = Dict(k => v/total_observed_kmers for (k,v) in kmer_weights)

In [ ]:
# function re-sample read, unanchored

In [46]:
read_length = length(FASTX.sequence(observation))

10

In [59]:
function sequence_to_canonical_kmers(kmer_type, sequence)
    return (BioSequences.canonical(kmer.fw) for kmer in BioSequences.each(kmer_type, sequence))
end

sequence_to_canonical_kmers (generic function with 1 method)

In [61]:
observation_kmers = collect(sequence_to_canonical_kmers(BioSequences.DNAMer{k}, FASTX.sequence(observation)))

8-element Vector{BioSequences.Mer{BioSequences.DNAAlphabet{2}, 3}}:
 CCG
 ACG
 GAC
 CGA
 CCG
 GCC
 AGC
 CAG

In [67]:
kmers = [observation_graph.vprops[v][:kmer] for v in LightGraphs.vertices(observation_graph)]
weights = [observation_graph.vprops[v][:weight] for v in LightGraphs.vertices(observation_graph)]

In [72]:
kmer = StatsBase.sample(kmers, StatsBase.weights(weights))
initial_orientation = true

true

In [ ]:
# walk forward until we've taken n steps or we run out of space

# determine remaining steps

# walk backward until we've taken n steps or we run out of space

In [ ]:
# function re-sample read, anchored on a kmer chosen at random given kmer probability weights

In [ ]:
# have it work on kmer graph rather than collapsed repeat graph

In [ ]:
# if working with collapsed repeat graph, we will have to start working with intra-node slicing of sequences
# because the unambiguous sequences are longer than the remaining length of the read overlapping that sequence

In [ ]:
# make a new graph from the re-sampled reads

In [ ]:
# compare the new graph to the error-free graph to the error-corrected graph